In [53]:
### changes for this commit
### facebook
### using replica v3 pricing db

In [54]:
import pandas as pd
import pencilbox as pb
from requests import get
import requests

In [55]:
con_rs = pb.get_connection("[Warehouse] Redshift")
con_rep = pb.get_connection("[Replica] Pricing V3")

In [56]:
sql = """

WITH MERCHANT AS
  (SELECT DISTINCT M.id,
                   m.name,
                   m.gr_id,
                   m.enabled_flag,
                   L.NAME AS city
   FROM lake_grofers_db.view_gr_merchant M
   INNER JOIN lake_grofers_db.gr_merchant_additional_info MAI ON MAI.MERCHANT_ID = M.ID
   AND (MAI.MERCHANT_STORE_TYPE = 'grofers')
   INNER JOIN lake_grofers_db.view_gr_locality L2 ON M.LOCALITY_ID = L2.ID
   AND (M.NAME NOT ILIKE '%%GROCERY%%MART%%'
        AND M.NAME NOT ILIKE '%%FRESHBURY%%'
        AND M.NAME NOT ILIKE '%%test%%'
        AND M.NAME NOT ILIKE '%%donation%%')
   INNER JOIN lake_grofers_db.view_gr_locality L1 ON L2.PARENT_LOCALITY_ID = L1.ID
   INNER JOIN lake_grofers_db.view_gr_locality L ON L1.PARENT_LOCALITY_ID = L.ID
   AND L.ID <> 2051 ),
   
   -- fetching mid name, city name and mid details
   
     mapping AS
  ( SELECT DISTINCT vmp.virtual_merchant_id::int AS frontend_id,
                    m1.gr_id AS frontend_gr_id,
                    m1.name AS frontend_name,
                    m1.city AS frontend_city,
                    vmp.real_merchant_id::int AS backend_id,
                    m.gr_id AS backend_gr_id,
                    m.name AS backend_name,
                    m.city AS backend_city,
                    co.outlet_id AS retail_outlet_id,
                    c.name AS retail_outlet_name,
                    x.name AS retail_outlet_city
   FROM lake_grofers_db.gr_virtual_to_real_merchant_mapping vmp
   INNER JOIN merchant m ON m.id=vmp.real_merchant_id
   INNER JOIN merchant m1 ON m1.id=vmp.virtual_merchant_id
   INNER JOIN lake_retail.console_outlet_cms_store co ON co.cms_store=vmp.real_merchant_id
   AND co.active=1
   INNER JOIN lake_retail.console_outlet c ON c.id=co.outlet_id
   LEFT JOIN lake_retail.console_location x ON x.id=c.tax_location_id
   AND m1.enabled_flag=TRUE
   AND vmp.enabled_flag=TRUE
   WHERE x.name IS NOT NULL
     AND x.name <>''
   ORDER BY m1.city ),
   
   -- fetching latest product images
   
     images AS (
     SELECT *,
          row_number() OVER (PARTITION BY product_id
                             ORDER BY update_ts DESC) AS rrank
   FROM lake_grofers_db.gr_product_sliding_image
   WHERE sort_order = 1 ),
   
   -- creating a base of all customers in grofers db
   
   base as (select dim_customer_key as customer_id, cart_checkout_ts_ist,merchant_id, merchant_name,dim_customer_address_key 
   from dwh.fact_sales_order_details fs join dwh.dim_merchant m on 
fs.dim_frontend_merchant_key = m.merchant_key where order_create_ts_ist between current_date-31 and current_Date-1),

-- listing all mids with min 5 orders in the past 2 days

mid_min_condition as (
select
frontend_merchant_id merchant_id,
count(distinct order_id) order_cnt from
dwh.fact_sales_order_item_details
--where order_create_ts_ist >= current_date-1 and order_current_status <> 'CANCELLED'
group by 1
--having order_cnt >= 5
),


 -- matching the most recent order with each mid

merc as (
select  merchant_name, a.merchant_id ,max(cart_checkout_ts_ist) as dt from 
base a
inner join
mid_min_condition b on a.merchant_id = b.merchant_id
group by 1,2),


-- lat long for merchants

merch_lat_long as (select
frontend_merchant_name frontend_name,
backend_merchant_name backend_name,
frontend_merchant_city_name frontend_city,
backend_merchant_city_name backend_city,
b.external_id merchant_id,
a.latitude,
a.longitude,
a.city
from
lake_logistics.logistics_node_address a
inner join
lake_logistics.logistics_node b on a.id = b.node_address_id
inner join 
dwh.dim_merchant_outlet_facility_mapping c on b.external_id = c.frontend_merchant_id
group by 1,2,3,4,5,6,7,8
),

-- matching each mid with users that had the most recent cart checkout and supplying his/ her lat long

SS as (
select 
m.*, b.customer_id,b.dim_customer_address_key, o.latitude location_lat, o.longitude location_lon, o.city
from 
merc m 
inner join 
base b on m.merchant_id = b.merchant_id and m.dt = b.cart_checkout_ts_ist
inner join 
merch_lat_long o on m.merchant_id = o.merchant_id
group by 1,2,3,4,5,6,7,8),

manufacture_info as (
select
a.product_id,
case when a.attribute_id = 632 then a.value else '' end manufacturer_info,
case when b.attribute_id = 641 then ltrim(rtrim(b.value)) else '' end origin_country,
case when d.attribute_id = 634 then ltrim(rtrim(d.value)) else '' end importer_name,
case when d.attribute_id = 634 then ltrim(rtrim(d.value)) else '' end importer_address,
c.Country_code
from 
lake_grofers_db.gr_product_attribute_mapping a
inner join 
lake_grofers_db.gr_product_attribute_mapping b on a.product_id = b.product_id
inner join
metrics.iso_codes c on ltrim(rtrim(b.value)) ilike c.country_name
inner join 
lake_grofers_db.gr_product_attribute_mapping d on a.product_id = d.product_id
where a.attribute_id in (632) and b.attribute_id in (641) and d.attribute_id in (634)
),

-- top selling items

top_selling as (
select product_id from
(select product_id, sum(product_quantity) sold from dwh.fact_sales_order_item_details where 
order_create_ts_ist >current_date-7 and order_current_status <> 'CANCELLED'
group by 1
order by 2 desc)
limit 2600
),


-- for each city, getting the highest price of the product and sharing the lat long of the latest order with that mid

final_agg as (
SELECT 
mpm.product_id as id,
replace(p.name, '%%',' percent')|| ' - '||p.unit as title,
p.name as description,
'http://grofers.s3.amazonaws.com/'||psi.image_path as image_link,
'android-app://com.grofers.customerapp/grofers/product?expr=%%22p'||p.id||'%%22&restricted=false' as mobile_android_app_link,
case when mpm.inventory_limit = 0 then 'out of stock'
when mpm.inventory_limit > 0 then 'in stock' end as availability,
mll.frontend_name AS frontend_name,
mll.merchant_id frontend_id,
--case when top_selling.product_id is not null then 'TS' else 'NTS' end  as custom_label_4,
p.brand as brand,
cat.name as google_product_category,
'new' as condition,
'no' as adult,
'no' as identifier_exists,
mll.frontend_city as custom_label_3,
'https://blinkit.com/prn/'||lower(replace(replace(p.name,' ','-'),'%%','percent'))||'/prid/'||p.id||'?lat='||location_lat||'&lon='||location_lon as link,
mpm.price as SP,
mpm.mrp,
mpm.product_id as item_group,
'' as custom_label_1,
'home'|| ' > '||cat.name|| ' > '||cat1.name|| ' > '||cat2.name as product_type,
lower(mll.frontend_city) as city,
location_lat as lat,
location_lon as lon,
'grofers://pdp?productId='||mpm.product_id as ios_url,
'grofers://pdp?productId='||mpm.product_id as android_url,
l0_category,
l1_category,
l2_category,
product_type ptype,
ss.merchant_id,
ss.location_lat,
ss.location_lon,
ss.city city_ss,
case when pam.Country_code is null then 'IN' else pam.Country_code end as origin_country,
case when pam.manufacturer_info  is null then 'HOT' else pam.manufacturer_info  end as manufacturer_info,
case when pam.Country_code is null then null when pam.Country_code = 'IN' then null else pam.importer_name end import_name,
case when pam.Country_code is null then null when pam.Country_code = 'IN' then null else pam.importer_address end importer_address,
row_number() over(partition by mll.frontend_city,mpm.product_id order by mpm.price desc ) as S_no
FROM 
lake_grofers_db.gr_merchant_product_mapping mpm
--left join
--top_selling on mpm.product_id = top_selling.product_id
INNER JOIN 
merch_lat_long mll on mll.merchant_id = mpm.merchant_id 
and mpm.enabled_flag=true and mpm.inventory_limit>0
INNER JOIN 
lake_grofers_db.gr_product p ON mpm.product_id = p.id--now mapping the category
inner join 
images psi on mpm.product_id = psi.product_id and rrank = 1
INNER JOIN 
lake_grofers_db.gr_product_category_mapping PCM ON P.ID = PCM.PRODUCT_ID
inner join 
ss on mll.merchant_id = ss.merchant_id
INNER JOIN 
lake_grofers_db.gr_category CAT2 ON PCM.CATEGORY_ID = CAT2.ID AND PCM.IS_PRIMARY=TRUE
INNER JOIN 
lake_grofers_db.gr_category CAT1 ON CAT2.PARENT_CATEGORY_ID = CAT1.ID
INNER JOIN 
lake_grofers_db.gr_category CAT ON CAT1.PARENT_CATEGORY_ID = CAT.ID and CAT.id not in (343,909)
inner join 
lake_rpc.item_product_mapping ipm on ipm.product_id=mpm.product_id 
inner join
dwh.dim_product dp on dp.product_id = mpm.product_id 
left join
manufacture_info pam on dp.product_id = pam.product_id
WHERE p.enabled_flag=TRUE
and dp.is_current = True
and psi.sort_order=1
and mpm.price>0
and mpm.mrp is not null
and mll.backend_name not ilike 'Super Store - B2B NCR Warehouse'
and mll.backend_city not ilike 'Haridwar'
and mll.Frontend_City not in ('%%Not in service area%%','durgapur')
and l0_category_id in ('1237','332','9','175')
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,mpm.update_ts)
   
Select
 item_group,
 id,
 title,
 description,
 link,
 image_link,
 availability,
 mrp||' INR' as price,
 SP||' INR' as sale_price,
 condition,
 brand,
 google_product_category,
 product_type,
 adult,
 l0_category as custom_label_0,
 l1_category as custom_label_1,
 ios_url,
 '960335206' as ios_app_store_id,
 'blinkit' as ios_app_name,
  android_url,
 'com.grofers.customerapp' as android_package,
 'blinkit' as android_app_name,
 frontend_id,
 'grofers://home' as mobile_link,
 link as icon_link
FROM final_agg
where S_no = 1
group by 1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25

"""

In [57]:
df = pd.read_sql(sql=sql, con=con_rs)
df.head()

,item_group,id,title,description,link,image_link,availability,price,sale_price,condition,...,custom_label_1,ios_url,ios_app_store_id,ios_app_name,android_url,android_package,android_app_name,frontend_id,mobile_link,icon_link
0,15286,15286,Coca-Cola Zero Soft Drink - 300 ml,Coca-Cola Zero Soft Drink,https://blinkit.com/prn/coca-cola-zero-soft-dr...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,38 INR,new,...,Soft Drinks,grofers://pdp?productId=15286,960335206,blinkit,grofers://pdp?productId=15286,com.grofers.customerapp,blinkit,30797,grofers://home,https://blinkit.com/prn/coca-cola-zero-soft-dr...
1,402565,402565,"Hershey's Kisses Almonds, Shaped Chocolates - ...","Hershey's Kisses Almonds, Shaped Chocolates",https://blinkit.com/prn/hershey's-kisses-almon...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,50 INR,43 INR,new,...,Chocolates,grofers://pdp?productId=402565,960335206,blinkit,grofers://pdp?productId=402565,com.grofers.customerapp,blinkit,30595,grofers://home,https://blinkit.com/prn/hershey's-kisses-almon...
2,35375,35375,Haldiram's Nagpur Aloo Bhujia - 150 g,Haldiram's Nagpur Aloo Bhujia,https://blinkit.com/prn/haldiram's-nagpur-aloo...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,40 INR,new,...,Bhujia & Mixtures,grofers://pdp?productId=35375,960335206,blinkit,grofers://pdp?productId=35375,com.grofers.customerapp,blinkit,29847,grofers://home,https://blinkit.com/prn/haldiram's-nagpur-aloo...
3,401228,401228,Bingo Mad Angles Masala Madness Crisps - Pack ...,Bingo Mad Angles Masala Madness Crisps - Pack ...,https://blinkit.com/prn/bingo-mad-angles-masal...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,34 INR,new,...,Chips & Crisps,grofers://pdp?productId=401228,960335206,blinkit,grofers://pdp?productId=401228,com.grofers.customerapp,blinkit,30756,grofers://home,https://blinkit.com/prn/bingo-mad-angles-masal...
4,456320,456320,Coolberg Strawberry Non-Alcoholic Beer - 330 ml,Coolberg Strawberry Non-Alcoholic Beer,https://blinkit.com/prn/coolberg-strawberry-no...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,79 INR,68 INR,new,...,Soft Drinks,grofers://pdp?productId=456320,960335206,blinkit,grofers://pdp?productId=456320,com.grofers.customerapp,blinkit,30841,grofers://home,https://blinkit.com/prn/coolberg-strawberry-no...


In [58]:
df.shape

(15512, 25)

In [59]:
### tobacco products

In [60]:
sql_tob = """

select product_id from
((select
product_id
from dwh.dim_product where is_current = true 
and lower(product_type) in ('paan','Cigarette','Hookah Flavor','Gulkand') )
union all
(select* from metrics.nicotine_products) ) group by 1

"""

df_tob = pd.read_sql(sql=sql_tob, con=con_rs)
df_tob.head()

,product_id
0,474056
1,193348
2,173120
3,352709
4,480934


In [61]:
df_tob_ll = df_tob.product_id.to_list()

In [62]:
df = df[(df.item_group.isin(df_tob_ll) == False)]
df.shape

(15512, 25)

In [63]:
sql = """SELECT cms_product_id item_group, cast(frontend_id as varchar) frontend_id, mrp actual_mrp, price actual_price FROM pricing_domain_pricerecommendation"""

In [64]:
df_prices = pd.read_sql(sql=sql, con=con_rep)
df_prices.head()

,item_group,frontend_id,actual_mrp,actual_price
0,315221,30985,115.0,115.0
1,478983,29867,60.0,57.0
2,32685,30900,68.0,68.0
3,429678,31330,150.0,139.0
4,95213,30900,199.0,191.0


In [65]:
merged_data = df.merge(df_prices, how="inner", on=["item_group", "frontend_id"])
merged_data.head()

,item_group,id,title,description,link,image_link,availability,price,sale_price,condition,...,ios_app_store_id,ios_app_name,android_url,android_package,android_app_name,frontend_id,mobile_link,icon_link,actual_mrp,actual_price
0,15286,15286,Coca-Cola Zero Soft Drink - 300 ml,Coca-Cola Zero Soft Drink,https://blinkit.com/prn/coca-cola-zero-soft-dr...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,38 INR,new,...,960335206,blinkit,grofers://pdp?productId=15286,com.grofers.customerapp,blinkit,30797,grofers://home,https://blinkit.com/prn/coca-cola-zero-soft-dr...,40.0,36.0
1,402565,402565,"Hershey's Kisses Almonds, Shaped Chocolates - ...","Hershey's Kisses Almonds, Shaped Chocolates",https://blinkit.com/prn/hershey's-kisses-almon...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,50 INR,43 INR,new,...,960335206,blinkit,grofers://pdp?productId=402565,com.grofers.customerapp,blinkit,30595,grofers://home,https://blinkit.com/prn/hershey's-kisses-almon...,50.0,41.0
2,35375,35375,Haldiram's Nagpur Aloo Bhujia - 150 g,Haldiram's Nagpur Aloo Bhujia,https://blinkit.com/prn/haldiram's-nagpur-aloo...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,40 INR,new,...,960335206,blinkit,grofers://pdp?productId=35375,com.grofers.customerapp,blinkit,29847,grofers://home,https://blinkit.com/prn/haldiram's-nagpur-aloo...,40.0,40.0
3,456320,456320,Coolberg Strawberry Non-Alcoholic Beer - 330 ml,Coolberg Strawberry Non-Alcoholic Beer,https://blinkit.com/prn/coolberg-strawberry-no...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,79 INR,68 INR,new,...,960335206,blinkit,grofers://pdp?productId=456320,com.grofers.customerapp,blinkit,30841,grofers://home,https://blinkit.com/prn/coolberg-strawberry-no...,79.0,68.0
4,5451,5451,Natural Tattva Rock Salt - 500 g,Natural Tattva Rock Salt,https://blinkit.com/prn/natural-tattva-rock-sa...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,55 INR,36 INR,new,...,960335206,blinkit,grofers://pdp?productId=5451,com.grofers.customerapp,blinkit,30775,grofers://home,https://blinkit.com/prn/natural-tattva-rock-sa...,55.0,41.0


In [66]:
merged_data.shape

(13985, 27)

In [67]:
merged_data["price"] = merged_data["actual_mrp"].astype(int).astype(str) + " INR"
merged_data["sale_price"] = merged_data["actual_price"].astype(int).astype(str) + " INR"
df = merged_data.copy()
df.head()

,item_group,id,title,description,link,image_link,availability,price,sale_price,condition,...,ios_app_store_id,ios_app_name,android_url,android_package,android_app_name,frontend_id,mobile_link,icon_link,actual_mrp,actual_price
0,15286,15286,Coca-Cola Zero Soft Drink - 300 ml,Coca-Cola Zero Soft Drink,https://blinkit.com/prn/coca-cola-zero-soft-dr...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,36 INR,new,...,960335206,blinkit,grofers://pdp?productId=15286,com.grofers.customerapp,blinkit,30797,grofers://home,https://blinkit.com/prn/coca-cola-zero-soft-dr...,40.0,36.0
1,402565,402565,"Hershey's Kisses Almonds, Shaped Chocolates - ...","Hershey's Kisses Almonds, Shaped Chocolates",https://blinkit.com/prn/hershey's-kisses-almon...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,50 INR,41 INR,new,...,960335206,blinkit,grofers://pdp?productId=402565,com.grofers.customerapp,blinkit,30595,grofers://home,https://blinkit.com/prn/hershey's-kisses-almon...,50.0,41.0
2,35375,35375,Haldiram's Nagpur Aloo Bhujia - 150 g,Haldiram's Nagpur Aloo Bhujia,https://blinkit.com/prn/haldiram's-nagpur-aloo...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,40 INR,new,...,960335206,blinkit,grofers://pdp?productId=35375,com.grofers.customerapp,blinkit,29847,grofers://home,https://blinkit.com/prn/haldiram's-nagpur-aloo...,40.0,40.0
3,456320,456320,Coolberg Strawberry Non-Alcoholic Beer - 330 ml,Coolberg Strawberry Non-Alcoholic Beer,https://blinkit.com/prn/coolberg-strawberry-no...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,79 INR,68 INR,new,...,960335206,blinkit,grofers://pdp?productId=456320,com.grofers.customerapp,blinkit,30841,grofers://home,https://blinkit.com/prn/coolberg-strawberry-no...,79.0,68.0
4,5451,5451,Natural Tattva Rock Salt - 500 g,Natural Tattva Rock Salt,https://blinkit.com/prn/natural-tattva-rock-sa...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,55 INR,41 INR,new,...,960335206,blinkit,grofers://pdp?productId=5451,com.grofers.customerapp,blinkit,30775,grofers://home,https://blinkit.com/prn/natural-tattva-rock-sa...,55.0,41.0


In [68]:
del df["frontend_id"]
del df["item_group"]
del df["actual_mrp"]
del df["actual_price"]
df.shape

(13985, 23)

In [69]:
### checking if all df is non empty
assert not (df.empty or df[df.availability == "in stock"].empty)

In [70]:
print(df.info(verbose=True))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13985 entries, 0 to 13984
Data columns (total 23 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   id                       13985 non-null  int64 
 1   title                    13985 non-null  object
 2   description              13985 non-null  object
 3   link                     13985 non-null  object
 4   image_link               13985 non-null  object
 5   availability             13985 non-null  object
 6   price                    13985 non-null  object
 7   sale_price               13985 non-null  object
 8   condition                13985 non-null  object
 9   brand                    13985 non-null  object
 10  google_product_category  13985 non-null  object
 11  product_type             13985 non-null  object
 12  adult                    13985 non-null  object
 13  custom_label_0           13985 non-null  object
 14  custom_label_1           13985 non-nul

In [71]:
print(df.shape)

(13985, 23)


In [72]:
### adding dag run flag condition

In [77]:
sheet_id = "1-uNp4c9uuGrWibaG2MHThepvx-o4Jctke3DpeS5Zsxg"
sheet_name = "flags"
flags_df = pb.from_sheets(sheet_id, sheet_name)
var = flags_df[flags_df.Name == "facebook_s3"]

In [78]:
var.Flag.to_list()[0] == "1"

True

In [79]:
df.head()

,id,title,description,link,image_link,availability,price,sale_price,condition,brand,...,custom_label_0,custom_label_1,ios_url,ios_app_store_id,ios_app_name,android_url,android_package,android_app_name,mobile_link,icon_link
0,15286,Coca-Cola Zero Soft Drink - 300 ml,Coca-Cola Zero Soft Drink,https://blinkit.com/prn/coca-cola-zero-soft-dr...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,36 INR,new,Coca-Cola,...,Cold Drinks & Juices,Soft Drinks,grofers://pdp?productId=15286,960335206,blinkit,grofers://pdp?productId=15286,com.grofers.customerapp,blinkit,grofers://home,https://blinkit.com/prn/coca-cola-zero-soft-dr...
1,402565,"Hershey's Kisses Almonds, Shaped Chocolates - ...","Hershey's Kisses Almonds, Shaped Chocolates",https://blinkit.com/prn/hershey's-kisses-almon...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,50 INR,41 INR,new,Hershey's Kisses,...,Sweet Tooth,Chocolates,grofers://pdp?productId=402565,960335206,blinkit,grofers://pdp?productId=402565,com.grofers.customerapp,blinkit,grofers://home,https://blinkit.com/prn/hershey's-kisses-almon...
2,35375,Haldiram's Nagpur Aloo Bhujia - 150 g,Haldiram's Nagpur Aloo Bhujia,https://blinkit.com/prn/haldiram's-nagpur-aloo...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,40 INR,40 INR,new,Haldiram's Nagpur,...,Munchies,Bhujia & Mixtures,grofers://pdp?productId=35375,960335206,blinkit,grofers://pdp?productId=35375,com.grofers.customerapp,blinkit,grofers://home,https://blinkit.com/prn/haldiram's-nagpur-aloo...
3,456320,Coolberg Strawberry Non-Alcoholic Beer - 330 ml,Coolberg Strawberry Non-Alcoholic Beer,https://blinkit.com/prn/coolberg-strawberry-no...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,79 INR,68 INR,new,Coolberg,...,Cold Drinks & Juices,Soft Drinks,grofers://pdp?productId=456320,960335206,blinkit,grofers://pdp?productId=456320,com.grofers.customerapp,blinkit,grofers://home,https://blinkit.com/prn/coolberg-strawberry-no...
4,5451,Natural Tattva Rock Salt - 500 g,Natural Tattva Rock Salt,https://blinkit.com/prn/natural-tattva-rock-sa...,http://grofers.s3.amazonaws.com/app/images/pro...,in stock,55 INR,41 INR,new,Natural Tattva,...,Organic & Gourmet,"Salt, Sugar & Honey",grofers://pdp?productId=5451,960335206,blinkit,grofers://pdp?productId=5451,com.grofers.customerapp,blinkit,grofers://home,https://blinkit.com/prn/natural-tattva-rock-sa...


In [80]:
df.shape

(13985, 23)

In [81]:
filename = "snapchat.csv"
filepath = f"/tmp/{filename}"
df.to_csv(filepath, index=False)
if var.Flag.to_list()[0] == "1":
    print("Running now")
    pb.to_s3(filepath, "grofers-facebook-product-feed", filename)
else:
    print("No runtime due to flag")  

Running now
